In [208]:
import os
import sys
import json

import pandas as pd
import numpy as np

from dotenv import load_dotenv
from googleapiclient.discovery import build

In [4]:
load_dotenv()

API_KEY = os.environ['YOUTUBE_API_KEY']
youtube = build('youtube', 'v3', developerKey=API_KEY)
# api = Api(api_key=API_KEY) 

In [177]:
request = youtube.channels().list(
        part="snippet,statistics,contentDetails",
        forUsername="Vsauce",
#         id='UCAov2BBv1ZJav0c_yHEciAw',
    )
response = request.execute()
print(json.dumps(response, indent=2))
print(response['items'][0]['snippet']['thumbnails']['high']['url'])

{
  "kind": "youtube#channelListResponse",
  "etag": "omWyCFJMFQpvqIIGT-E4GARm3QU",
  "pageInfo": {
    "totalResults": 1,
    "resultsPerPage": 5
  },
  "items": [
    {
      "kind": "youtube#channel",
      "etag": "o8dhZ1_-yzlMj-9smhpjeCVBx1Q",
      "id": "UC6nSFpj9HTCZ5t-N3Rm3-HA",
      "snippet": {
        "title": "Vsauce",
        "description": "Our World is Amazing. \n\nQuestions? Ideas? Tweet me: http://www.twitter.com/tweetsauce\n\nVsauce was created by Michael Stevens in the summer of 2010.\n\nVsauce is...\n\nMichael Stevens: Producer/Host of Vsauce1\nKevin Lieber: Producer/Host of Vsauce2\nJake Roper: Producer/Host of Vsauce3\nEric Langlay: VFX for Vsauce1/2/3\nYou: Thanks for watching!!!",
        "customUrl": "vsauce1",
        "publishedAt": "2007-07-30T20:43:33Z",
        "thumbnails": {
          "default": {
            "url": "https://yt3.ggpht.com/ytc/AAUvwnhZ3RdTd90CWLjszcugYGMU4I72zJAVkphAfSflTQ=s88-c-k-c0x00ffffff-no-rj",
            "width": 88,
            

In [178]:
upload_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
upload_id

'UU6nSFpj9HTCZ5t-N3Rm3-HA'

In [69]:
request = youtube.playlistItems().list(
        part="snippet",
        playlistId=upload_id,
        maxResults=50,
    )
response = request.execute()
response

{'kind': 'youtube#playlistItemListResponse',
 'etag': 'QHqT5BkxaghauMvbyl4d4PsLz5g',
 'items': [{'kind': 'youtube#playlistItem',
   'etag': 'CP7RXLE8ZVffhiatxOEuhvKQQSI',
   'id': 'VVVJOU40VEFVWDhrX29MOWk3MlVZQWJ3LkRPdmw5OXlDQVNR',
   'snippet': {'publishedAt': '2021-01-11T05:54:16Z',
    'channelId': 'UCI9N4TAUX8k_oL9i72UYAbw',
    'title': 'Hand and Brain, Maybe Odds ?! Ft.Sahil (Editor),Angad (Conqueror) and Rakesh Bhai',
    'description': 'Connect with me on social media:\nhttps://www.instagram.com/adhibanchess/\nhttps://twitter.com/adhibanchess\nhttps://discord.gg/WKd3qPe\nThumbnail Credit: Airton Alonso\n\n\nMusic credit: Saggian, https://open.spotify.com/artist/6dQdP4BL7WaOUGQGPdpDYo?nd=1\n\nFor Business Enquiries/Sponsors Contact on:\nteam@adhibanbaskaran.me',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/DOvl99yCASQ/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/DOvl99yCASQ/mqdefault.jpg',
      'width': 3

In [70]:
response['items'][0]['snippet']['title']

'Hand and Brain, Maybe Odds ?! Ft.Sahil (Editor),Angad (Conqueror) and Rakesh Bhai'

In [70]:
next_page_token = response.get('nextPageToken', 0)
while next_page_token:
    next_request = youtube.playlistItems().list_next(
        request, response
    )

    next_response = next_request.execute()
    print(json.dumps(next_response, indent=2))
    break

    next_page_token = next_response.get('nextPageToken', 0)
    request, response = next_request, next_response

In [71]:
next_page = response.get('nextPageToken', 0)
next_page

0

In [72]:
request = youtube.videos().list(
        part="statistics, contentDetails",
        id="8ifjYxkMhUg,SP8hdGSN-0k",
        maxResults=1,
    )
response = request.execute()

print(json.dumps(response, indent=2))

{
  "kind": "youtube#videoListResponse",
  "etag": "OSvjznhgUfDxKIXjbU9LKYHyevA",
  "items": [
    {
      "kind": "youtube#video",
      "etag": "sbLH1xNQLTWmyz8oswVP5KuW3Qs",
      "id": "8ifjYxkMhUg",
      "contentDetails": {
        "duration": "PT8M38S",
        "dimension": "2d",
        "definition": "hd",
        "caption": "false",
        "licensedContent": true,
        "contentRating": {},
        "projection": "rectangular"
      },
      "statistics": {
        "viewCount": "203433",
        "likeCount": "22665",
        "dislikeCount": "318",
        "favoriteCount": "0",
        "commentCount": "753"
      }
    },
    {
      "kind": "youtube#video",
      "etag": "pAOvm0o7BxwVzssHYQ-yb5-Dgic",
      "id": "SP8hdGSN-0k",
      "contentDetails": {
        "duration": "PT1H27M46S",
        "dimension": "2d",
        "definition": "hd",
        "caption": "false",
        "licensedContent": false,
        "contentRating": {},
        "projection": "rectangular"
      },


In [188]:
COLUMNS = [
    'vid_id', 'vid_title', 'vid_description', 'thumbnail_url', 'upload_time', 
    'duration', 'views', 'likes', 'dislikes', 'comment_counts',
]

def add_video_data(res):
    data = []
    for items in res['items']:
        video_id = items['snippet']['resourceId']['videoId']
        video_title = items['snippet']['title']
        video_desc = items['snippet']['description']
        upload_time = items['snippet']['publishedAt']
        thumbnail_url = items['snippet']['thumbnails']['high']['url']

        # print(video_title, upload_time, thumbnail_url)
        temp = [video_id, video_title, video_desc, thumbnail_url, upload_time] + [''] * 5
        data.append(temp)
    
    return data


def get_video_stats(res):
    data = []

    for items in res['items']:
        duration = items['contentDetails']['duration']
        views = items['statistics'].get('viewCount', 0)
        likes = items['statistics'].get('likeCount', 0)
        dislikes = items['statistics'].get('dislikeCount', 0)
        comment_count = items['statistics'].get('commentCount', 0)
        
        temp = [duration, views, likes, dislikes, comment_count]
        data.append(temp)

    return data

In [189]:
all_data = []
video_stats = []

request = youtube.playlistItems().list(
    part="snippet",
    playlistId=upload_id,
    maxResults=50,
)
response = request.execute()

data = add_video_data(response)
all_data.extend(data)

next_page_token = response.get('nextPageToken', 0)

while next_page_token:
    next_request = youtube.playlistItems().list_next(request, response)
    next_response = next_request.execute()

    data = add_video_data(next_response)
    all_data.extend(data)

    next_page_token = next_response.get('nextPageToken', 0)
    request, response = next_request, next_response

raw = pd.DataFrame(all_data, columns=COLUMNS)

In [190]:
raw

,vid_id,vid_title,vid_description,thumbnail_url,upload_time,duration,views,likes,dislikes,comment_counts
0,zHL9GP_B30E,Illusions of Time,Use code BRAINS to get free shipping within th...,https://i.ytimg.com/vi/zHL9GP_B30E/hqdefault.jpg,2021-01-08T07:54:28Z,,,,,
1,U6VBV4QUMu0,The Odd Number Rule,Subscribe to our Curiosity Box! https://www.cu...,https://i.ytimg.com/vi/U6VBV4QUMu0/hqdefault.jpg,2020-07-29T17:18:49Z,,,,,
2,egEraZP9yXQ,How Many Holes Does a Human Have?,Now you're thinking: https://www.curiositybox....,https://i.ytimg.com/vi/egEraZP9yXQ/hqdefault.jpg,2020-02-17T08:40:53Z,,,,,
3,9Vmwsg8Eabo,What Is The Scariest Thing?,Everyone is scared of something ... but what's...,https://i.ytimg.com/vi/9Vmwsg8Eabo/hqdefault.jpg,2019-10-16T23:45:32Z,,,,,
4,b0cakXwiZ8s,I Watch 3 Episodes of Mind Field With Our Expe...,EVERY EPISODE OF MIND FIELD IS NOW FREE for th...,https://i.ytimg.com/vi/b0cakXwiZ8s/hqdefault.jpg,2019-10-10T23:09:16Z,,,,,
...,...,...,...,...,...,...,...,...,...,...
364,m_icISNqoS8,Mark and Michael - Vsauce BLOOPERS,Mark Douglas and Michael Stevens talking about...,https://i.ytimg.com/vi/m_icISNqoS8/hqdefault.jpg,2010-06-30T23:17:10Z,,,,,
365,4R1WPK3u8II,Mario Farts!,Mario Farts when a mushroom upsets his stomach...,https://i.ytimg.com/vi/4R1WPK3u8II/hqdefault.jpg,2010-06-29T18:14:40Z,,,,,
366,UaOeE8AJD9Q,iPhone 4 is for LOSERS? -- Wackygamer,Jeff and Adam debate the iPhone 4. \r\n\r\nLe...,https://i.ytimg.com/vi/UaOeE8AJD9Q/hqdefault.jpg,2010-06-26T17:49:23Z,,,,,
367,GXDKhCWUY8k,Killzone Trailer PARODY Commentary -- Wackygamer,Subscribe to Vsauce! http://www.youtube.com/su...,https://i.ytimg.com/vi/GXDKhCWUY8k/hqdefault.jpg,2010-06-24T22:39:24Z,,,,,


In [191]:
video_stats = []
total_videos = raw.shape[0]
print(total_videos)

for i in range(0, total_videos, 50):
    request = youtube.videos().list(
            part="statistics, contentDetails",
            id=','.join(raw['vid_id'].values[i:i+50]),
        )
    
    response = request.execute()

#     print(json.dumps(response, indent=2))
    try:
        data = get_video_stats(response)
    except Exception as e:
        print(e)
        print(json.dumps(response, indent=2))
# print(data)
    video_stats.extend(data)

print(len(video_stats))

369
369


In [192]:
raw.iloc[:, 5:] = video_stats

In [193]:
raw

,vid_id,vid_title,vid_description,thumbnail_url,upload_time,duration,views,likes,dislikes,comment_counts
0,zHL9GP_B30E,Illusions of Time,Use code BRAINS to get free shipping within th...,https://i.ytimg.com/vi/zHL9GP_B30E/hqdefault.jpg,2021-01-08T07:54:28Z,PT31M8S,4450658,475190,2375,45122
1,U6VBV4QUMu0,The Odd Number Rule,Subscribe to our Curiosity Box! https://www.cu...,https://i.ytimg.com/vi/U6VBV4QUMu0/hqdefault.jpg,2020-07-29T17:18:49Z,PT22M7S,5913919,373042,8083,37297
2,egEraZP9yXQ,How Many Holes Does a Human Have?,Now you're thinking: https://www.curiositybox....,https://i.ytimg.com/vi/egEraZP9yXQ/hqdefault.jpg,2020-02-17T08:40:53Z,PT21M27S,8676631,489335,8083,56200
3,9Vmwsg8Eabo,What Is The Scariest Thing?,Everyone is scared of something ... but what's...,https://i.ytimg.com/vi/9Vmwsg8Eabo/hqdefault.jpg,2019-10-16T23:45:32Z,PT40M41S,7700699,314621,5230,42419
4,b0cakXwiZ8s,I Watch 3 Episodes of Mind Field With Our Expe...,EVERY EPISODE OF MIND FIELD IS NOW FREE for th...,https://i.ytimg.com/vi/b0cakXwiZ8s/hqdefault.jpg,2019-10-10T23:09:16Z,PT3H16S,4854750,57776,3036,5596
...,...,...,...,...,...,...,...,...,...,...
364,m_icISNqoS8,Mark and Michael - Vsauce BLOOPERS,Mark Douglas and Michael Stevens talking about...,https://i.ytimg.com/vi/m_icISNqoS8/hqdefault.jpg,2010-06-30T23:17:10Z,PT1M18S,294706,4378,135,843
365,4R1WPK3u8II,Mario Farts!,Mario Farts when a mushroom upsets his stomach...,https://i.ytimg.com/vi/4R1WPK3u8II/hqdefault.jpg,2010-06-29T18:14:40Z,PT50S,9571772,54910,6714,7146
366,UaOeE8AJD9Q,iPhone 4 is for LOSERS? -- Wackygamer,Jeff and Adam debate the iPhone 4. \r\n\r\nLe...,https://i.ytimg.com/vi/UaOeE8AJD9Q/hqdefault.jpg,2010-06-26T17:49:23Z,PT4M13S,159776,1475,235,526
367,GXDKhCWUY8k,Killzone Trailer PARODY Commentary -- Wackygamer,Subscribe to Vsauce! http://www.youtube.com/su...,https://i.ytimg.com/vi/GXDKhCWUY8k/hqdefault.jpg,2010-06-24T22:39:24Z,PT1M1S,158256,1307,200,308


In [194]:
df = raw.copy()

In [195]:
INT_COLS = ['views', 'likes', 'dislikes', 'comment_counts'] 

df[INT_COLS] = df[INT_COLS].astype(int)

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   vid_id           369 non-null    object
 1   vid_title        369 non-null    object
 2   vid_description  369 non-null    object
 3   thumbnail_url    369 non-null    object
 4   upload_time      369 non-null    object
 5   duration         369 non-null    object
 6   views            369 non-null    int64 
 7   likes            369 non-null    int64 
 8   dislikes         369 non-null    int64 
 9   comment_counts   369 non-null    int64 
dtypes: int64(4), object(6)
memory usage: 29.0+ KB


In [197]:
import datetime

datetime.datetime.strptime('2019-06-26T09:29:17Z', "%Y-%m-%dT%H:%M:%S%z")

datetime.datetime(2019, 6, 26, 9, 29, 17, tzinfo=datetime.timezone.utc)

In [198]:
df['upload_date'] = df['upload_time'].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").day
)
df['upload_month'] = df['upload_time'].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").month
)
df['upload_year'] = df['upload_time'].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").year
)
df['upload_hour'] = df['upload_time'].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").hour
)
df['upload_minute'] = df['upload_time'].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").minute
)
df['upload_second'] = df['upload_time'].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").second
)

In [199]:
dur = 'PT1H58M45S'

def convert_dur(s):
    mapp = { 'H': '*60', 'M': '*1', 'S': '/60' }
    time, val, units, prev_char = 0, [], [], True
    for char in s[2:]:
        if char.isdigit():
            if prev_char:
                val.append(char)
            else:
                val[-1] += char
            prev_char = False
        else:
            units.append(char)
            prev_char = True
           
    for u, v in zip(units, val):
        time += eval(v+mapp[u])
        
    return time

In [200]:
convert_dur(dur)

118.75

In [201]:
df['vid_dur'] = df['duration'].apply(lambda x: convert_dur(x))

In [202]:
CATEGORY_COLS = ['upload_year', 'upload_month', 'upload_date', 'upload_hour', 'upload_minute', 'upload_second'] 

df[CATEGORY_COLS] = df[CATEGORY_COLS].astype('category')

In [215]:
df['likes_percent'] = df['likes'] / (df['likes'] + df['dislikes'])
df['dislikes_percent'] = df['dislikes'] / (df['likes'] + df['dislikes'])

df['likes_percent'].fillna(0, inplace=True)
df['dislikes_percent'].fillna(0, inplace=True)

In [216]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   vid_id            369 non-null    object  
 1   vid_title         369 non-null    object  
 2   vid_description   369 non-null    object  
 3   thumbnail_url     369 non-null    object  
 4   upload_time       369 non-null    object  
 5   duration          369 non-null    object  
 6   views             369 non-null    int64   
 7   likes             369 non-null    int64   
 8   dislikes          369 non-null    int64   
 9   comment_counts    369 non-null    int64   
 10  upload_date       369 non-null    category
 11  upload_month      369 non-null    category
 12  upload_year       369 non-null    category
 13  upload_hour       369 non-null    category
 14  upload_minute     369 non-null    category
 15  upload_second     369 non-null    category
 16  vid_dur           369 non-

In [212]:
df.loc[df['likes_percent'].isnull()]

,vid_id,vid_title,vid_description,thumbnail_url,upload_time,duration,views,likes,dislikes,comment_counts,upload_date,upload_month,upload_year,upload_hour,upload_minute,upload_second,vid_dur,likes_percent,dislikes_percent
6,AgbeGFYluEA,Mind Reading,In this episode I visit a researcher who is st...,https://i.ytimg.com/vi/AgbeGFYluEA/hqdefault.jpg,2018-12-14T10:56:08Z,PT22M22S,0,0,0,4612,14,12,2018,10,56,8,22.366667,NaN,NaN
7,BR-eMMCp7tg,Behavior and Belief,"How can a lie become true? In this episode, Dr...",https://i.ytimg.com/vi/BR-eMMCp7tg/hqdefault.jpg,2018-10-25T21:55:22Z,PT28M27S,0,0,0,4405,25,10,2018,21,55,22,28.450000,NaN,NaN
8,xna-kdXZQHQ,How to Talk to Aliens,Are we alone in the universe? Even if we could...,https://i.ytimg.com/vi/xna-kdXZQHQ/hqdefault.jpg,2018-12-14T10:49:23Z,PT26M7S,0,0,0,8652,14,12,2018,10,49,23,26.116667,NaN,NaN
9,zRxI0DaQrag,Should I Die?,"If I could live forever, should I? How does be...",https://i.ytimg.com/vi/zRxI0DaQrag/hqdefault.jpg,2018-11-29T18:04:46Z,PT31M5S,0,0,0,11420,29,11,2018,18,4,46,31.083333,NaN,NaN
11,rA5qnZUXcqo,The Stilwell Brain,There are 100 billion individual neurons in th...,https://i.ytimg.com/vi/rA5qnZUXcqo/hqdefault.jpg,2018-11-28T16:09:04Z,PT26M32S,0,0,0,2937,28,11,2018,16,9,4,26.533333,NaN,NaN
12,yZlOw3rsPBM,Moral Licensing,How are our moral decisions influenced by fact...,https://i.ytimg.com/vi/yZlOw3rsPBM/hqdefault.jpg,2018-11-27T20:53:59Z,PT28M28S,0,0,0,4843,27,11,2018,20,53,59,28.466667,NaN,NaN
15,NXNGvDdkXZE,The Electric Brain - Mind Field S2 (Ep 8),Our nervous system is fundamentally electric. ...,https://i.ytimg.com/vi/NXNGvDdkXZE/hqdefault.jpg,2017-12-08T00:14:16Z,PT27M44S,0,0,0,0,8,12,2017,0,14,16,27.733333,NaN,NaN
16,wcPzTr-BbAA,Divergent Minds - Mind Field S2 (Ep 7),Scientists can’t dissect living people’s brain...,https://i.ytimg.com/vi/wcPzTr-BbAA/hqdefault.jpg,2017-12-08T00:10:49Z,PT26M13S,0,0,0,0,8,12,2017,0,10,49,26.216667,NaN,NaN
17,QDCcuCHOIyY,The Power of Suggestion - Mind Field S2 (Ep 6),Exactly how do placebos work? Researchers bel...,https://i.ytimg.com/vi/QDCcuCHOIyY/hqdefault.jpg,2017-12-08T00:08:01Z,PT28M49S,0,0,0,0,8,12,2017,0,8,1,28.816667,NaN,NaN
18,JMpuxLIgjPs,How to Make a Hero - Mind Field S2 (Ep 5),"What makes a hero? Dr. Philip Zimbardo, the ps...",https://i.ytimg.com/vi/JMpuxLIgjPs/hqdefault.jpg,2017-11-30T00:00:54Z,PT30M16S,0,0,0,0,30,11,2017,0,0,54,30.266667,NaN,NaN


In [217]:
import plotly.express as px

LABELS = { 
    'upload_year': 'Year', 
    'views': 'Total Views', 
    'vid_dur': 'Video Length (mins)',
    'vid_title': 'Video Title',
    'dislikes_percent': 'Dislike Percentage',
    'likes_percent': 'Like Percentage',
    'vid_id': 'Total Videos',
}

# views vs video length (like percent)

fig = px.scatter(
    data_frame=df,
    x='vid_dur',
    y='views',
    color='upload_year',
    size='likes_percent',
    opacity=0.4,
    hover_data=['vid_title'],
    labels=LABELS,
    
)
fig.add_hline(y=df['views'].median(), line_dash='dash', )

fig.show()

In [176]:
# views vs video length (dislike percent)

fig = px.scatter(
    data_frame=df,
    x='vid_dur',
    y='views',
    color='upload_year',
    size='dislikes_percent',
    opacity=0.6,
    hover_data=['vid_title'],
    labels=LABELS,
    
)
fig.add_hline(y=df['views'].median(), line_dash='dash', )

fig.show()

In [92]:
df

,vid_id,vid_title,vid_description,thumbnail_url,upload_time,duration,views,likes,dislikes,comment_counts,upload_date,upload_month,upload_year,upload_hour,upload_minute,upload_second,vid_dur,likes_percent,dislikes_percent
0,DOvl99yCASQ,"Hand and Brain, Maybe Odds ?! Ft.Sahil (Editor...",Connect with me on social media:\nhttps://www....,https://i.ytimg.com/vi/DOvl99yCASQ/hqdefault.jpg,2021-01-11T05:54:16Z,PT1H58M45S,13181,1513,10,11,11,1,2021,5,54,16,118.750000,0.993434,0.006566
1,8wFn9__z2XY,"Chill Stream, Puzzle Rush Survival, Blitz,Bull...",Connect with me on social media:\nhttps://www....,https://i.ytimg.com/vi/8wFn9__z2XY/hqdefault.jpg,2021-01-08T12:12:46Z,PT2H27M10S,12032,1439,10,5,8,1,2021,12,12,46,147.166667,0.993099,0.006901
2,uA2La0xm0-4,Chill stream & Chess with subs (Discord edition),Connect with me on social media:\nhttps://www....,https://i.ytimg.com/vi/uA2La0xm0-4/hqdefault.jpg,2021-01-06T13:43:37Z,PT2H52M54S,7436,1105,8,8,6,1,2021,13,43,37,172.900000,0.992812,0.007188
3,qgsKtdmx8UU,Bday Stream. Part3,Connect with me on social media:\nhttps://www....,https://i.ytimg.com/vi/qgsKtdmx8UU/hqdefault.jpg,2021-01-05T10:51:57Z,PT1H23M30S,6218,838,6,5,5,1,2021,10,51,57,83.500000,0.992891,0.007109
4,Fsk0dKO-QFc,"Hand & Brain,Bughouse Ft.@VAIBHAV SETHIA,@Anir...",Link to Join Graphy and everyone gets a one mo...,https://i.ytimg.com/vi/Fsk0dKO-QFc/hqdefault.jpg,2020-12-04T17:10:38Z,PT2H36M45S,48215,3300,39,34,4,12,2020,17,10,38,156.750000,0.988320,0.011680
5,jrw7ywJbR1o,Secret Training Episode 1: The Beast and the G...,Connect with me on social media:\nhttps://www....,https://i.ytimg.com/vi/jrw7ywJbR1o/hqdefault.jpg,2020-11-30T18:03:10Z,PT2H13M10S,42256,3319,27,10,30,11,2020,18,3,10,133.166667,0.991931,0.008069
6,0g5BeE_Pgzo,Streaming Until Radja reaches 50k Subs. Ft.Sou...,Subscribe to Radja Channel :https://www.youtub...,https://i.ytimg.com/vi/0g5BeE_Pgzo/hqdefault.jpg,2020-11-30T11:39:11Z,PT48M,30892,2671,142,24,30,11,2020,11,39,11,48.000000,0.949520,0.050480
7,8VrW5d7ZkCQ,"Hand & Brain Match, OBS Adventures, Raiding Ga...",Connect with me on social media:\nhttps://www....,https://i.ytimg.com/vi/8VrW5d7ZkCQ/hqdefault.jpg,2020-11-25T15:06:03Z,PT3H11M7S,139692,9899,107,54,25,11,2020,15,6,3,191.116667,0.989306,0.010694
8,lLFh3C1cXxo,Ryzen's Immortals. Ft. Ryzen (Join Discord),Connect with me on social media:\nhttps://www....,https://i.ytimg.com/vi/lLFh3C1cXxo/hqdefault.jpg,2020-11-23T10:59:28Z,PT1H43M50S,27576,2371,32,31,23,11,2020,10,59,28,103.833333,0.986683,0.013317
9,gogKiUodeFM,"CELEBRATION STREAM of winning #C0B3 Ft.Joel,Va...",Connect with me on social media:\nhttps://www....,https://i.ytimg.com/vi/gogKiUodeFM/hqdefault.jpg,2020-11-21T16:12:09Z,PT1H22M55S,130406,12589,72,122,21,11,2020,16,12,9,82.916667,0.994313,0.005687


In [116]:
yearwise = df.groupby('upload_year').count()['vid_id']

In [120]:
fig = px.line(
    data_frame=yearwise,
    y='vid_id',
    labels=LABELS
)

fig.update_layout(
    xaxis = dict(
        dtick = 1
    )
)

fig.show()